# Word2Vec for Tamil words. 

Converting words to vector representation and similarity map generation

Fine plural form of words


In [1]:
import pandas as pd
import numpy as np 
import re
from matplotlib import pyplot as plt
import regex
from tqdm import tqdm


In [2]:
import sys

In [3]:
sys.setrecursionlimit(2500)

In [4]:
In_Colab = False
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    In_Colab = True

In [5]:
if In_Colab:
    from google.colab import drive
    drive.mount('/gdrive')

    %cd /gdrive/My\ Drive/Tamizh/data/wiki_data

In [6]:
from constants import *

In [7]:
data = pd.read_csv("../data/tawiki_data_df.csv")

In [8]:
# Shape of the data
data.shape

(247759, 1)

In [9]:
data = data.dropna()

In [10]:
data.shape

(211814, 1)

#### Convert each sentence to list of tokens. 

*observation : this have more then 2 crore vocabulary 
* Need to find the root words  - 

In [11]:
data.head()

,Content
0,விக்கிப்பீடியா மொழிகள்
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...


### Drop NA values

### Convert as one long sentence. This will be useful for cleaning vallina migu sorrkal.

## Data Cleaning 

In [12]:
def split_text(text):
    return regex.findall(r'\X', text, regex.U)

def conv_base(text):
    return [split_map.get(k) for k in split_text(text)]

def _lastmap(text):
    if len(text[-1])>1:
        return text[-1][-1]
    return text[-1][0]

def _firstmap(text):
    return text[0][0]

def is_vallmigu_pair(text1,text2):
    try:
        text1 = conv_base(text1)
        text2 = conv_base(text2)
        if (_lastmap(text1) in vall_migu) and (_firstmap(text2) in vall_migu):
            return True
        else:
            return False
    except:
        return False

In [13]:
def clean_split(text):
    text = re.sub("\s+"," ",text.strip())
    return text.split(' ')

In [14]:
data.head()

,Content
0,விக்கிப்பீடியா மொழிகள்
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...


In [15]:
vallmigu_pair_set = set()

In [16]:
data['processed'] = data.Content.apply(clean_split)

In [17]:
def parse_clean_text(text):
    global vallmigu_pair_set
    for i in range(len(text)-2):
        if is_vallmigu_pair(text[i],text[i+1]):
            vallmigu_pair_set.add((text[i],text[i+1]))
            text[i] = ''.join(split_text(text[i])[:-1])
    return text

In [18]:
# from multiprocessing import Pool 

In [19]:
# with Pool(5) as p:
#     text_list = p.map(parse_clean_text,data.Content.values)

In [20]:

for row in tqdm(range(data.shape[0])):
    text = data.processed.iloc[row]
    for i in range(len(text)-2):
        if is_vallmigu_pair(text[i],text[i+1]):
            vallmigu_pair_set.add((text[i],text[i+1]))
            text[i] = ''.join(split_text(text[i])[:-1])
    data.processed.iloc[row]= text



100%|██████████| 211814/211814 [15:36<00:00, 226.09it/s]


In [21]:
# ~1200 sec nearly 20 mins 

In [22]:
data.head()

,Content,processed
0,விக்கிப்பீடியா மொழிகள்,"[விக்கிப்பீடியா, மொழிகள்]"
1,இங்கு தமிழ் விக்கிப்பீடியாவைப் பற்றிய உங்கள் ப...,"[இங்கு, தமிழ், விக்கிப்பீடியாவை, பற்றிய, உங்கள..."
2,விக்கிப்பீடியா அதன் வாசகர்களால் ஒருமித்து எழுத...,"[விக்கிப்பீடியா, அதன், வாசகர்களால், ஒருமித்து,..."
3,விக்கிப்பீடியாவுக்கு கட்டுரை எழுதி மேம்படுத்தி...,"[விக்கிப்பீடியாவுக்கு, கட்டுரை, எழுதி, மேம்படு..."
4,முகலாயக் கட்டிடக்கலைமுகலாய கட்டடக்கலையின் சிற...,"[முகலாய, கட்டிடக்கலைமுகலாய, கட்டடக்கலையின், சி..."


In [23]:
len(vallmigu_pair_set)

787020

#Word2Vec

Generate word2vec representation for tokens in tamil. 



In [24]:
import pickle

with open('../data/vallmigu_pair.pkl','wb') as f:
    pickle.dump(vallmigu_pair_set,f)

In [25]:
#Generate word 2 vec

from gensim.models import Word2Vec

In [26]:
texts = data.processed.values

In [27]:
 
model = Word2Vec(texts, vector_size=100, window=50, min_count=10, workers=6)

In [28]:
# len(model.wv.vocab)

In [34]:
model.wv.most_similar("வயல்")

[('வயல்கள்', 0.759262204170227),
 ('பனை', 0.7393096685409546),
 ('மரங்களும்', 0.7370815277099609),
 ('தென்னை', 0.734723687171936),
 ('மருத', 0.7170229554176331),
 ('வயலும்', 0.7027963399887085),
 ('சூழ்ந்த', 0.6919956207275391),
 ('காடும்', 0.6733906269073486),
 ('வேம்பு', 0.6708276271820068),
 ('வயல்களும்', 0.6685636639595032)]

In [31]:
model.save("../model/tawiki_w2v")